In [38]:
import os
import numpy as np
import pandas as pd

In [39]:
def add_load_weighted(hourly_load_zonal, load_ratio_zone, load_bus_zone, 
                      num_load_bus_tot, zone_ids, num_load_bus_zone):
    num_hours = hourly_load_zonal.shape[0]
    zone_load_bus = np.empty(len(zone_ids), dtype=object)
    load_bus_idx = np.zeros(num_load_bus_tot)
    load_bus_load = np.zeros((num_load_bus_tot, num_hours))
    n = 0
    for i, zone_id in enumerate(zone_ids):
        zone_load_tot = hourly_load_zonal[zone_id].values
        zone_load_bus[i] = np.outer(load_ratio_zone[i], zone_load_tot)
        load_bus_idx[n:n+num_load_bus_zone[i]] = load_bus_zone[i]
        load_bus_load[n:n+num_load_bus_zone[i], :] = zone_load_bus[i]
        n += num_load_bus_zone[i]

    bus_wload = pd.DataFrame(load_bus_load.T, columns=load_bus_idx,
                             index=hourly_load_zonal.index)
    bus_wload = bus_wload.sort_index(axis=1)
    bus_wload.columns = bus_wload.columns.astype(int)

    return bus_wload


In [40]:
# Read NYS bus table
filename = '/mnt/Bo_HDD/NYgrid-python/data/grid/npcc_new.csv'
bus_info = pd.read_csv(filename)
nys_bus = bus_info[~bus_info['zone'].isnull()]
nys_bus_wload = nys_bus[nys_bus['sumLoadP0'] > 0]
nys_bus_wload

,idx,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,sumGenP0,sumGenQ0,sumLoadP0,sumLoadQ0
38,39,LEEDS,345.0,1.1,0.9,1.04100,0.057140,-74.001,42.042,2,G,0.00000,0.00000,1.00,0.000
40,41,ROTTERDAM,115.0,1.1,0.9,1.02634,-0.058833,-73.780,43.086,2,F,0.00000,0.00000,7.00,1.250
41,42,ALBANY,115.0,1.1,0.9,1.04000,-0.044794,-73.628,42.702,2,F,3.70000,1.34949,1.77,0.170
44,45,PORTER,115.0,1.1,0.9,1.04349,0.068892,-75.240,43.131,2,E,0.00000,0.00000,2.56,0.170
45,46,COLTON,115.0,1.1,0.9,1.04781,0.107949,-75.051,44.428,2,E,0.00000,0.00000,0.23,-0.425
46,47,MOSES W,230.0,1.1,0.9,1.03000,0.203842,-75.211,44.654,2,E,4.00000,-0.03268,5.28,1.000
48,49,PLATTSBURGH,115.0,1.1,0.9,0.96523,-0.019481,-74.134,44.699,2,D,0.00000,0.00000,2.08,0.170
50,51,CLAY,115.0,1.1,0.9,1.04000,0.148847,-76.261,42.939,2,C,1.41000,0.20685,7.31,0.750
51,52,ROCHESTER,345.0,1.1,0.9,1.04193,0.338821,-77.482,43.312,2,B,0.00000,0.00000,1.95,0.000
52,53,ROCHESTER,115.0,1.1,0.9,1.03000,0.314252,-77.777,42.889,2,B,8.33000,1.73977,7.28,2.500


In [41]:
# Load bus and ratio calculation
zone_ids = nys_bus['zone'].unique()
load_bus_zone = np.empty(11, dtype=object)
load_ratio_zone = np.empty(11, dtype=object)
num_load_bus_zone = np.zeros(11, dtype=int)

for i, zone_id in enumerate(zone_ids):
    load_bus_table = nys_bus_wload[nys_bus_wload['zone'] == zone_id]
    load_bus_zone[i] = load_bus_table['idx'].values
    if load_bus_table.shape[0] > 0:
        load_ratio_zone[i] = (load_bus_table['sumLoadP0'] / 
                              load_bus_table['sumLoadP0'].sum()).values
    else:
        load_bus_zone[i] = nys_bus[nys_bus['zone'] == zone_id]['idx'].values
        load_ratio_zone[i] = np.ones(len(load_bus_zone[i])) / len(load_bus_zone[i])
    num_load_bus_zone[i] = len(load_bus_zone[i])

num_load_bus_tot = sum(num_load_bus_zone)


In [42]:
# Read load data
filename = '/media/by276/Bo_SSD/offshore_wind/Data/Demand/demand_base_zone.csv'
zonal_demand = pd.read_csv(filename, index_col=0, parse_dates=True)
zonal_demand


,F,C,I,B,G,E,H,D,A,J,K
TimeStamp,,,,,,,,,,,
2002-01-01 00:00:00,1225.0000,1809.0000,491.0000,1027.0000,1193.0000,829.0000,264.0000,762.0000,1600.0000,5360.468452,2174.531548
2002-01-01 01:00:00,1162.0000,1653.0000,453.0000,984.0000,1117.0000,783.0000,244.0000,758.0000,1543.0000,5072.513945,2051.486055
2002-01-01 02:00:00,1110.0000,1513.0000,431.0000,935.0000,1060.0000,744.0000,223.0000,750.0000,1494.0000,4786.285282,1936.714718
2002-01-01 03:00:00,1101.0000,1501.0000,422.0000,904.0000,1031.0000,720.0000,217.0000,748.0000,1437.0000,4602.290232,1867.709768
2002-01-01 04:00:00,1105.0000,1493.0000,404.0000,905.0000,1027.0000,725.0000,215.0000,748.0000,1428.0000,4506.175631,1844.824369
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,1513.1182,1936.9957,715.7587,1186.0897,1232.7810,1026.6797,411.9453,675.9467,1782.7536,5721.312900,2611.661600
2020-12-31 20:00:00,1457.0589,1874.7353,692.8507,1146.1326,1188.3394,997.9535,393.8276,665.3796,1744.5531,5543.351500,2506.094500
2020-12-31 21:00:00,1393.7715,1828.9972,671.4122,1103.5253,1146.6135,953.3583,370.6922,653.3130,1695.8337,5368.983700,2397.669700


In [43]:
# Distribute load to buses
sample_hour_load = add_load_weighted(zonal_demand, load_ratio_zone,
                                     load_bus_zone, num_load_bus_tot,
                                     zone_ids, num_load_bus_zone)
sample_hour_load

,39,41,42,45,46,47,49,51,52,53,...,73,74,75,76,77,78,79,80,81,82
TimeStamp,,,,,,,,,,,,,,,,,,,,,
2002-01-01 00:00:00,238.60000,977.765108,247.234892,262.978934,23.627014,542.394052,762.0000,854.802198,216.971831,810.028169,...,238.60000,264.0000,119.30000,119.30000,477.20000,491.0000,931.942092,1242.589456,2680.234226,2680.234226
2002-01-01 01:00:00,223.40000,927.480046,234.519954,248.386617,22.315985,512.297398,758.0000,781.087912,207.887324,776.112676,...,223.40000,244.0000,111.70000,111.70000,446.80000,453.0000,879.208309,1172.277746,2536.256972,2536.256972
2002-01-01 02:00:00,212.00000,885.974914,224.025086,236.014870,21.204461,486.780669,750.0000,714.934066,197.535211,737.464789,...,212.00000,223.0000,106.00000,106.00000,424.00000,431.0000,830.020593,1106.694125,2393.142641,2393.142641
2002-01-01 03:00:00,206.20000,878.791334,222.208666,228.401487,20.520446,471.078067,748.0000,709.263736,190.985915,713.014085,...,206.20000,217.0000,103.10000,103.10000,412.40000,422.0000,800.447044,1067.262725,2301.145116,2301.145116
2002-01-01 04:00:00,205.40000,881.984036,223.015964,229.987608,20.662949,474.349442,748.0000,705.483516,191.197183,713.802817,...,205.40000,215.0000,102.70000,102.70000,410.80000,404.0000,790.639015,1054.185354,2253.087816,2253.087816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,246.55620,1207.734025,305.384175,325.687736,29.261008,671.730956,675.9467,915.283682,250.582331,935.507369,...,246.55620,411.9453,123.27810,123.27810,493.11240,715.7587,1119.283543,1492.378057,2860.656450,2860.656450
2020-12-31 20:00:00,237.66788,1162.988860,294.070040,316.575088,28.442293,652.936119,665.3796,885.863933,242.140690,903.991910,...,237.66788,393.8276,118.83394,118.83394,475.33576,692.8507,1074.040500,1432.054000,2771.675750,2771.675750
2020-12-31 21:00:00,229.32270,1112.474401,281.297099,302.428407,27.171302,623.758590,653.3130,864.251424,233.139148,870.386152,...,229.32270,370.6922,114.66135,114.66135,458.64540,671.4122,1027.572729,1370.096971,2684.491850,2684.491850


In [45]:
sample_hour_load.to_csv('/media/by276/Bo_SSD/offshore_wind/Data/Demand/demand_base_bus.csv')